In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import warnings; warnings.simplefilter('ignore', FutureWarning)
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import pymongo
from pymongo import MongoClient
from avian_vocalizations.evaluator import ParamSpace, EvaluatorFactory
from hyperopt.mongoexp import MongoTrials, MongoCtrl, MongoJobs, MongoWorker
from hyperopt import hp, fmin, tpe
from vastai.api import VastClient
from avian_vocalizations.toolbench import print_module_versions
print_module_versions(globals())

Using TensorFlow backend.


             IPython  {'7.9.0'}
          tensorflow  {'1.15.0'}
             pymongo  {'3.9.0'}
 avian_vocalizations  {'0.0.2'}
            hyperopt  {'0.2.2'}


## Local Hyoeropt Training

In [3]:
from hyperopt import Trials

local_trials = Trials()

hp_space = ParamSpace(n_frames=128, 
                      dropout_rate=hp.uniform('dropout',0,.5),
                      batch_size=hp.choice('batch_size',[64]),
                      audio_feature_type='stacked', # Can be 'spectrogram', mfcc', 'both', or 'stacked'
                     )

evaluator = EvaluatorFactory(n_splits=1, n_epochs=2, data_dir="../data")

best = fmin(evaluator, hp_space, trials=local_trials, algo=tpe.suggest, max_evals=1)

Running training trial with 1 splits, 2 epochs, with hyperparams: ParamSpace(n_frames=128, dropout_rate=0.01963793777982853, batch_size=64, audio_feature_type='stacked')
Split 1/1                                             
Split 1: min loss: 4.53819, accuracy at min loss: 0.01099, min loss at epoch 0
Cross Validation Accuracy: mean(val_acc[argmin(val_loss)]): 0.0110 +/-0.0000, mean loss: 4.5382 +/-0.0000
100%|██████████| 1/1 [01:20<00:00, 80.73s/trial, best loss: 4.53818941116333]


In [4]:
local_trials.attachments

{'ATTACH::0::params': b'{"n_frames": 128, "dropout_rate": 0.01963793777982853, "batch_size": 64, "audio_feature_type": "stacked"}',
 'ATTACH::0::summary': b'Model: "model_1"\n_________________________________________________________________\nLayer (type)                 Output Shape              Param #   \n=================================================================\nstacked (InputLayer)         (None, 128, 128, 1)       0         \n_________________________________________________________________\nconv2d_1 (Conv2D)            (None, 126, 126, 64)      640       \n_________________________________________________________________\nmax_pooling2d_1 (MaxPooling2 (None, 42, 42, 64)        0         \n_________________________________________________________________\ndropout_1 (Dropout)          (None, 42, 42, 64)        0         \n_________________________________________________________________\nconv2d_2 (Conv2D)            (None, 40, 40, 64)        36928     \n_____________________

In [12]:
hp_space

ParamSpace(n_frames=128, dropout_rate=<hyperopt.pyll.base.Apply object at 0x7fb124365780>, batch_size=<hyperopt.pyll.base.Apply object at 0x7fb124365f60>)

## Remote Hyperopt Worker

In [31]:
vast = VastClient().authenticate()

Initializing vast.ai client with api_key from /home/sam/.vast_api_key.


In [12]:
vast.instances[-1].wait_until_running()

464626: running   running              $0.1131/hr ssh4.vast.ai:24626  650.7↑ 608.5↓  flops:13.698048T
	1X 10.9GB GTX 1080 Ti, 5.333328/16X 31.8GB Xeon® E5-2630 v3 
	Successfully loaded tensorflow/tensorflow:1.15.2-gpu-py3-jupyter

In [32]:
vast.get_instances()[-1]

465113: running   running              $0.1131/hr ssh4.vast.ai:25113  652.3↑ 597.7↓  flops:13.698048T
	1X 10.9GB GTX 1080 Ti, 5.333328/16X 31.8GB Xeon® E5-2630 v3 
	Successfully loaded tensorflow/tensorflow:1.15.2-gpu-py3-jupyter

In [49]:
instance = vast.get_running_instances()[-1]
# instance.start()
remote = instance.pb_remote
instance.ssh_connection_command

'ssh root@ssh5.vast.ai -p 25268 -i /home/sam/.ssh/vastai'

## Update code repo

In [6]:
with remote.cwd('/root/avian_vocalizations'):
    print(remote['git']('reset','--hard','HEAD^'))
    print(remote['git']('pull'))
    print(remote['git']('status'))

HEAD is now at 9cbb633 tensorflow 1.15.0. Added hyperopt trial reporting.

Updating 9cbb633..5b6daa0
Fast-forward
 .gitignore                                         |   2 +
 Hyperopt.ipynb                                     | 556 ------------
 HyperoptTrials.ipynb                               | 297 -------
 Run Hyperopt Job.ipynb                             | 486 -----------
 avian_vocalizations/__init__.py                    |   2 +-
 avian_vocalizations/data.py                        |  21 +-
 avian_vocalizations/evaluator.py                   |   5 +-
 avian_vocalizations/visualization.py               |   1 +
 db_backup/file_ids.json                            |   0
 db_backup/fs.chunks.json                           |  46 +
 db_backup/fs.files.json                            |  56 ++
 db_backup/job_ids.json                             |   1 +
 db_backup/jobs.json                                |  11 +
 download_tensorboard_logs.sh                       |   4 +
 hyperopt_db_expo

## Run some trials

In [57]:
tunnel = instance.get_tunnel(27019, 27017)

In [59]:
trials.delete_all()

In [60]:
trials = MongoTrials('mongo://localhost:27019/avian_vocalizations/jobs', 
                     exp_key='alpha1')
hp_space = ParamSpace(n_frames=128, 
                      dropout_rate=hp.uniform('dropout',0,.5),
                      batch_size=hp.choice('batch_size',[64]),
                     )

evaluator = EvaluatorFactory(n_splits=2, n_epochs=200)

best = fmin(evaluator, hp_space, trials=trials, algo=tpe.suggest, max_evals=20)
best

  0%|          | 0/20 [10:06<?, ?trial/s, best loss: ?]


AutoReconnect: localhost:27019: [Errno 111] Connection refused

In [ ]:
trials.trial_attachments(trials.best_trial).items()

In [ ]:
# trials.delete_all()

## Cleanup

In [24]:
tunnel.close()

In [ ]:
# instance.stop()